## Imports

In [183]:
import numpy as np
from qutip import about, basis, rand_ket, tensor
from qutip_qip.operations import Measurement
from qutip_qip.operations.gates import gate_sequence_product
from qutip_qip.qasm import read_qasm, print_qasm

Random circuits: https://docs.quantum.ibm.com/api/qiskit/circuit_random#random-circuits

In [184]:
#make a random quantum circuit with qiskit

from qiskit.circuit.random import random_clifford_circuit
 
qiskit_circ = random_clifford_circuit(num_qubits=8, num_gates=100)
qiskit_circ.draw()

┌───┐                                    »
q_0: ────────■────────────────┤ X ├────────────────────────────────────»
     ┌─────┐ │  ┌───┐         └─┬─┘                                    »
q_1: ┤ Sdg ├─┼──┤ I ├───────────┼────────────────────────────■──────X──»
     └─────┘ │ ┌┴───┴─┐         │                            │      │  »
q_2: ────────┼─┤0     ├───■─────┼────────────────────────────┼──────X──»
             │ │      │ ┌─┴─┐   │  ┌──────┐                  │         »
q_3: ────────┼─┤      ├─┤ X ├───■──┤1     ├──────────────────┼─────────»
             │ │  Ecr │ └───┘      │      │       ┌──────┐ ┌─┴─┐       »
q_4: ────────┼─┤      ├────────────┤      ├───────┤0     ├─┤ Y ├───────»
      ┌───┐  │ │      │┌─────┐┌───┐│  Ecr │┌─────┐│  Ecr │┌┴───┴─┐     »
q_5: ─┤ I ├──┼─┤1     ├┤ Sdg ├┤ H ├┤      ├┤ Sdg ├┤1     ├┤1     ├─────»
      └───┘  │ └──────┘└─────┘└───┘│      │└┬───┬┘└──────┘│  Ecr │┌───┐»
q_6: ────────┼─────────────────────┤0     ├─┤ I ├─────────┤0     ├┤ Y ├»
             │                     └──────┘ └───┘         └──────┘└───┘»
q_7: ────────■─────────────────────────────────────────────────────────»
                                                                       »
«                                ┌────┐ ┌────┐             ┌──────┐┌───┐»
«q_0: ────■───────────────────■──┤ √X ├─┤ √X ├─────────────┤0     ├┤ I ├»
«         │     ┌───┐  ┌───┐  │  ├───┬┘ └────┘             │      │└───┘»
«q_1: ────■─────┤ Y ├──┤ I ├──■──┤ S ├─────────────────────┤      ├─────»
«       ┌────┐  ├───┤  └───┘     └───┘                     │      │     »
«q_2: ──┤ √X ├──┤ I ├──────────────────────────────────────┤      ├─────»
«       └────┘  └───┘                                      │  Dcx │     »
«q_3: ────────────────────────■────────────────────────────┤      ├─────»
«               ┌───┐  ┌───┐  │ ┌──────┐┌───┐ ┌───┐┌──────┐│      │     »
«q_4: ──────────┤ X ├──┤ H ├──┼─┤ √Xdg ├┤ S ├─┤ Y ├┤ √Xdg ├┤      ├─────»
«     ┌────────┐└─┬─┘  ├───┤  │ └──────┘└───┘ └───┘└──────┘│      │┌───┐»
«q_5: ┤1       ├──┼────┤ H ├──┼────────────────────────────┤1     ├┤ X ├»
«     │        │  │   ┌┴───┴┐ │  ┌───┐                     └──────┘└───┘»
«q_6: ┤  Iswap ├──■───┤ Sdg ├─┼──┤ S ├────X─────────────────────────────»
«     │        │┌────┐└─────┘ │  └───┘    │   ┌───┐                     »
«q_7: ┤0       ├┤ √X ├────────■───────────X───┤ X ├─────────────────────»
«     └────────┘└────┘                        └───┘                     »
«             ┌────────┐     ┌──────┐                                       »
«q_0: ────────┤0       ├─────┤1     ├─────────────────────■───────────────■─»
«             │        │     │      │     ┌───┐           │               │ »
«q_1: ────────┤        ├─────┤      ├──■──┤ H ├───────────┼───────────────┼─»
«             │        │     │      │  │  └───┘ ┌───┐     │    ┌────────┐ │ »
«q_2: ────────┤        ├─────┤      ├──┼────■───┤ X ├─────■────┤1       ├─┼─»
«             │        │     │      │  │    │  ┌┴───┴─┐┌──────┐│        │ │ »
«q_3: ────────┤        ├─────┤  Dcx ├──┼────┼──┤1     ├┤0     ├┤        ├─┼─»
«             │  Iswap │     │      │  │    │  │      ││  Dcx ││        │ │ »
«q_4: ────────┤        ├─────┤      ├──┼────┼──┤  Dcx ├┤1     ├┤  Iswap ├─┼─»
«     ┌──────┐│        │┌───┐│      │  │    │  │      │├──────┤│        │ │ »
«q_5: ┤0     ├┤        ├┤ Y ├┤      ├──■────■──┤0     ├┤1     ├┤        ├─┼─»
«     │  Ecr ││        │└───┘│      │┌───┐┌───┐└──────┘│      ││        │ │ »
«q_6: ┤1     ├┤        ├─────┤0     ├┤ X ├┤ Y ├────────┤  Ecr ├┤0       ├─┼─»
«     └──────┘│        │     └──────┘└───┘└───┘        │      │└────────┘ │ »
«q_7: ────────┤1       ├───────────────────────────────┤0     ├───────────■─»
«             └────────┘                               └──────┘             »
«                     ┌────────┐┌────┐        ┌───┐                     »
«q_0: ────────────────┤1       ├┤ √X ├────────┤ X ├─────────────────────»
«                     │        │└────┘        └─┬─┘      

Convert to qasm via Qiskit: https://docs.quantum.ibm.com/api/qiskit/qasm2#exporting-examples

In [185]:
import qiskit.qasm2
from qiskit.circuit import QuantumCircuit
 
print(qiskit.qasm2.dumps(qiskit_circ))

OPENQASM 2.0;
include "qelib1.inc";
gate rzx(param0) q0,q1 { h q1; cx q0,q1; rz(param0) q1; cx q0,q1; h q1; }
gate ecr q0,q1 { rzx(pi/4) q0,q1; x q0; rzx(-pi/4) q0,q1; }
gate iswap q0,q1 { s q0; s q1; h q0; cx q0,q1; cx q1,q0; h q1; }
gate dcx q0,q1 { cx q0,q1; cx q1,q0; }
qreg q[8];
id q[5];
ecr q[2],q[5];
sdg q[1];
cx q[2],q[3];
cz q[7],q[0];
cx q[3],q[0];
sdg q[5];
h q[5];
ecr q[6],q[3];
id q[6];
sdg q[5];
ecr q[4],q[5];
id q[1];
cy q[1],q[4];
swap q[2],q[1];
ecr q[6],q[5];
y q[6];
iswap q[7],q[5];
cz q[0],q[1];
y q[1];
cx q[6],q[4];
id q[1];
sx q[2];
sx q[7];
cz q[0],q[1];
sx q[0];
sdg q[6];
h q[4];
h q[5];
cz q[7],q[3];
sx q[0];
s q[6];
swap q[6],q[7];
dcx q[0],q[5];
id q[2];
s q[1];
id q[0];
x q[5];
x q[7];
sxdg q[4];
ecr q[5],q[6];
s q[4];
iswap q[0],q[7];
y q[4];
y q[5];
dcx q[6],q[0];
cz q[5],q[1];
cz q[2],q[5];
x q[2];
x q[6];
cz q[2],q[0];
y q[6];
dcx q[5],q[3];
sxdg q[4];
h q[1];
dcx q[3],q[4];
iswap q[6],q[2];
ecr q[7],q[5];
cz q[7],q[0];
sxdg q[7];
dcx q[4],q[2];
iswap q[

In [186]:
#Now add it to a file for clarity

qiskit.qasm2.dump(qiskit_circ, "random_circuit.qasm")
path = "random_circuit.qasm"

## QASM Completed ^

## QASM Conversion 

### Imports

In [187]:
import numpy as np
from qutip import about, basis, rand_ket, tensor
from qutip_qip.operations import Measurement
from qutip_qip.operations.gates import gate_sequence_product
from qutip_qip.qasm import read_qasm, print_qasm

### Read in the file to a string

In [188]:
with open(path, 'r') as file:
    qasm_str = file.read()

In [189]:
type(qasm_str)

str

### Append basic gate definitions to the string

For more definitions, see this repo: https://github.com/HQSquantumsimulations/qoqo_qasm/blob/aff4aba03ae12e12599ed11b98c9b7ecaef0c021/roqoqo-qasm/src/interface.rs#L1345

In [190]:
# Insert gate definitions after the 'include' line in qasm_str

swap_gate_def = [
    'gate swap a, b{',
    'cx b, a;',
    'cx a, b;',
    'cx b, a;',
    '}'
]

sx_gate_def = [
    'gate sx a {',
    'u1(-pi/2) a;',
    'u2(0,pi) a;',
    'u1(-pi/2) a;',
    '}'
]

sxdg_gate_def = [
    'gate sxdg a {',
    'u1(pi/2) a;',
    'u2(0,pi) a;',
    'u1(pi/2) a;',
    '}'
]

lines = qasm_str.splitlines()
for i, line in enumerate(lines):
    if line.strip().startswith('include'):
        insert_idx = i + 1
        break

gate_defs = swap_gate_def + [''] + sx_gate_def + [''] + sxdg_gate_def + ['']
lines[insert_idx:insert_idx] = gate_defs
qasm_str = '\n'.join(lines)

### Review the string using the built-in qasm to qutip conversion tools

In [191]:
qutip_circ = read_qasm(qasm_str, strmode=True)
gate_sequence_product(qutip_circ.propagators())

Quantum object: dims=[[2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2]], shape=(256, 256), type='oper', dtype=CSR, isherm=False
Qobj data =
[[-0.04419417+0.04419417j  0.04419417-0.04419417j -0.04419417+0.04419417j
  ... -0.04419417+0.04419417j -0.04419417+0.04419417j
   0.04419417-0.04419417j]
 [ 0.04419417-0.04419417j  0.04419417-0.04419417j -0.04419417+0.04419417j
  ... -0.04419417+0.04419417j -0.04419417+0.04419417j
  -0.04419417+0.04419417j]
 [ 0.04419417+0.04419417j  0.04419417+0.04419417j  0.04419417+0.04419417j
  ... -0.04419417-0.04419417j  0.04419417+0.04419417j
   0.04419417+0.04419417j]
 ...
 [ 0.04419417+0.04419417j  0.04419417+0.04419417j -0.04419417-0.04419417j
  ...  0.04419417+0.04419417j  0.04419417+0.04419417j
   0.04419417+0.04419417j]
 [ 0.04419417-0.04419417j  0.04419417-0.04419417j  0.04419417-0.04419417j
  ...  0.04419417-0.04419417j -0.04419417+0.04419417j
  -0.04419417+0.04419417j]
 [ 0.04419417-0.04419417j -0.04419417+0.04419417j -0.04419417+0.04419417j
  ..